In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [3]:
path = "../training/conf/experiment/cnn_htr_char_lines.yaml"

In [4]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [5]:
cfg

{'defaults': [{'override /mapping': None}, {'override /criterion': None}, {'override /datamodule': None}, {'override /network': None}, {'override /model': None}, {'override /lr_schedulers': None}, {'override /optimizers': None}], 'criterion': {'_target_': 'text_recognizer.criterions.label_smoothing.LabelSmoothingLoss', 'smoothing': 0.1, 'ignore_index': 1000}, 'mapping': {'_target_': 'text_recognizer.data.word_piece_mapping.WordPieceMapping', 'num_features': 1000, 'tokens': 'iamdb_1kwp_tokens_1000.txt', 'lexicon': 'iamdb_1kwp_lex_1000.txt', 'data_dir': None, 'use_words': False, 'prepend_wordsep': False, 'special_tokens': ['<s>', '<e>', '<p>']}, 'callbacks': {'stochastic_weight_averaging': {'_target_': 'pytorch_lightning.callbacks.StochasticWeightAveraging', 'swa_epoch_start': 0.8, 'swa_lrs': 0.05, 'annealing_epochs': 10, 'annealing_strategy': 'cos', 'device': None}}, 'optimizers': {'madgrad': {'_target_': 'madgrad.MADGRAD', 'lr': 0.0001, 'momentum': 0.9, 'weight_decay': 0, 'eps': 1e-06,

In [7]:
net = instantiate(cfg.network)

In [8]:
from torchinfo import summary

In [14]:
summary(net, list(map(lambda x: list(x), cfg.summary)), device="cpu", depth=1)

Layer (type:depth-idx)                             Output Shape              Param #
ConvTransformer                                    --                        --
├─EfficientNet: 1-1                                [1, 1280, 1, 32]          7,142,272
├─Sequential: 1-2                                  [1, 128, 32]              163,968
├─Embedding: 1-3                                   [1, 89, 128]              128,768
├─PositionalEncoding: 1-4                          [1, 89, 128]              --
├─Decoder: 1-5                                     [1, 89, 128]              13,179,264
├─Linear: 1-6                                      [1, 89, 1006]             129,774
Total params: 20,744,046
Trainable params: 20,744,046
Non-trainable params: 0
Total mult-adds (M): 715.11
Input size (MB): 0.23
Forward/backward pass size (MB): 184.97
Params size (MB): 82.98
Estimated Total Size (MB): 268.17